This notebook should examine the merged data under "/data/merged".
- "Semester" column has different values.
- "Program Code" can be converted from float to integer.
- "Program Short Name" can be interpolated, i.e., filled with IS for 901.
- "Course Name" may have string literals, which must be omitted.
- "Instructor Name" encoding must be fixed.
- "Instructor Title" should either be dropped or merged with "Instructor Name".
- "Course Objectives" has meaningless keywords that must be removed, e.g. "toO1:"
- "Prerequisite" has "Set 1:" keyword that must be removed.

In [1]:
import os
import pandas as pd
import glob
import json
import re
import numpy as np

In [2]:
DATA_PATH = os.path.join("..", "data")
MERGED_DATA_PATH = os.path.join(DATA_PATH, "merged")
CLEANED_DATA_PATH = os.path.join(DATA_PATH, "cleaned")
CSV_SEPARATOR = "|"

In [3]:
PROGRAMS = {
    901 : {"Short Name" : "IS",     "Long Name" : "INFORMATION SYSTEMS"},
    902 : {"Short Name" : "COGS",   "Long Name" : "COGNITIVE SCIENCES"},
    903 : {"Short Name" : "MS",     "Long Name" : "MODELLING AND SIMULATION"},
    904 : {"Short Name" : "ION",    "Long Name" : "INFORMATICS ONLINE"},
    905 : {"Short Name" : "SM",     "Long Name" : "SOFTWARE MANAGEMENT"},
    906 : {"Short Name" : "MI",     "Long Name" : "MEDICAL INFORMATICS"},
    908 : {"Short Name" : "BIN",    "Long Name" : "BIOINFORMATICS"},
    909 : {"Short Name" : "MMI",    "Long Name" : "MULTIMEDIA INFORMATICS"},
    910 : {"Short Name" : "CSEC",   "Long Name" : "CYBER SECURITY"},
    911 : {"Short Name" : "DI",     "Long Name" : "DATA INFORMATICS"}
}

In [4]:
merged_files = os.listdir(MERGED_DATA_PATH)
merged_files

['902-merged.csv',
 '905-merged.csv',
 '908-merged.csv',
 '911-merged.csv',
 '903-merged.csv',
 '906-merged.csv',
 '910-merged.csv',
 '901-merged.csv',
 '904-merged.csv',
 '909-merged.csv']

In [5]:
file = os.path.join(MERGED_DATA_PATH, merged_files[0])

df = pd.read_csv(file, sep=CSV_SEPARATOR)
df

,Semester,Program Code,Program Short Name,Course Code,Course Name,Credit,ECTS Credit,Course Section,Capacity,Day1,Start Hour1,End Hour1,Instructor Name,Instructor Title,Language of Instruction,Level of Study,Course Objectives,Course Learning Outcomes,Course Content,Prerequisite
0,2024-2025 Fall,902.0,COGS,9020501,ALGORITHMIC STRUCTURES IN COGNITION,3.0,8.0,1.0,25.0,Wednesday,14:40,17:30,UMUT ÖZGE,Assist.Prof.Dr,English,Graduate,PLEASE VISIT THE COURSE WEBSITE AND COME TO TH...,"By the end of the semester, a successful stude...",Natural language and linguistic knowledge. Lan...,Not Available
1,2024-2025 Fall,902.0,COGS,9020507,COGNITIVE SCIENCE PRIMER I,1.0,5.0,1.0,25.0,Monday,12:40,13:30,MURAT PERİT ÇAKIR,Assoc.Prof.Dr.,English,Graduate,NaN,NaN,The course covers topics which are considered ...,Not Available
2,2024-2025 Fall,902.0,COGS,9020515,ARTIFICIAL INTELLIGENCE FOR COGNITIVE SCIENCE,3.0,8.0,1.0,15.0,Wednesday,08:40,11:30,BARBAROS YET,Assoc.Prof.Dr.,English,Graduate,"At the end of this course, the students will k...",The student will have learned the techniques f...,Fundamental Techniques of Artificial Intellige...,Not Available
3,2024-2025 Fall,902.0,COGS,9020532,THEORETICAL LINGUISTICS,3.0,8.0,1.0,25.0,Thursday,14:40,17:30,UMUT ÖZGE,Assist.Prof.Dr,English,Graduate,PLEASE VISIT THE COURSE WEBSITE AND COME TO TH...,By the end of the course students are expected...,"A survey of the history of linguistics, sound-...",Not Available
4,2024-2025 Fall,902.0,COGS,9020536,RESEARCH METHODS AND STATISTICS FOR COGNITIVE ...,3.0,8.0,1.0,15.0,Tuesday,11:40,14:30,MURAT PERİT ÇAKIR,Assoc.Prof.Dr.,English,Graduate,The major objective of this course is to enabl...,At the end of the course students will be able...,Research methods: The students will be introdu...,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Fall Semesters.,NaN,NaN,9020520,ADVANCED RESEARCH ON LINGUISTICS IN COGNITIVE ...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,Graduate,NaN,NaN,The course offers advanced research in linguis...,Not Available
120,Fall Semesters.,NaN,NaN,9020516,INTRODUCTION TO PROBABILISTIC PROGRAMMING,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,Graduate,This course will examine the basic principles ...,Upon completion of this course (a) the student...,Introduction to Probabilistic Programming; gen...,Not Available
121,Spring Semesters.,NaN,NaN,9020541,LANGUAGE ACQUISITION,NaN,8.0,NaN,NaN,NaN,NaN,NaN,Prof.Dr. DENÝZ ZEYREK BOZÞAHÝN,NaN,English,Graduate,NaN,NaN,The course aims to examine the theories and re...,Not Available
122,Fall or Spring Semesters.,NaN,NaN,9020557,SITUATED AND DISTRIBUTED COGNITION,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,Graduate,By the end of the course students will explore...,This course will allow students to learn about...,this course will introduce topics that are gai...,Not Available


In [6]:
def correct_turkish_characters(text: str):
    # Define a mapping of incorrect characters to correct Turkish characters
    
    character_map = {
        'Ý': 'İ',
        'Þ': 'Ş',
        'ð': 'ğ',
        'þ': 'ş',
        'ý': 'ı',
        'Ð': 'Ğ'
    }

    # Apply the replacements
    for old_char, new_char in character_map.items():
        text = text.replace(old_char, new_char)

    return text

def get_code(filename: str):
    return int(re.search(r'\d+', filename).group())

In [7]:
for file in merged_files:
    path = os.path.join(MERGED_DATA_PATH, file)
    code = get_code(path)

    df = pd.read_csv(path, sep=CSV_SEPARATOR)

    df["Program Code"] = code
    df["Program Short Name"] = PROGRAMS[code]["Short Name"]
    df["Program Long Name"] = PROGRAMS[code]["Long Name"]
    
    
    
    df["Credit"] = df["Credit"].fillna(0)
    df["Credit"] = df["Credit"].astype(int)
    
    df["ECTS Credit"] = df["ECTS Credit"].astype(str)
    df["ECTS Credit"] = df["ECTS Credit"].str.replace(",",".")
    df["ECTS Credit"] = df["ECTS Credit"].astype(float).astype(int)
    
    df["Course Section"] = df["Course Section"].fillna(0)
    df["Course Section"] = df["Course Section"].astype(int)
    
    df["Capacity"] = df["Capacity"].fillna(0)
    df["Capacity"] = df["Capacity"].astype(int)
    
    
    # Fix encoding
    # Prepend "Instructor Title" with "Instructor Name"
    df["Instructor Name"] = df["Instructor Name"].fillna("")
    df["Instructor Name"] = df["Instructor Name"].str.replace("STAFF", "Not Available")
    df["Instructor Name"] = df["Instructor Name"].apply(correct_turkish_characters)
    df["Instructor Title"] = df["Instructor Title"].fillna("")
    df["Instructor Title"] = df["Instructor Title"].str.replace("-", "")
    df["Instructor Name"] = df["Instructor Title"] + " " + df["Instructor Name"]
    df["Instructor Name"] = df["Instructor Name"].apply(lambda x: x[1:] if x[0]== ' ' else x)
    df["Instructor Name"] = df["Instructor Name"].fillna("Not Available")
    
    df = df.drop(columns=['Instructor Title'])
    
    
    df["Day1"] = df["Day1"].fillna("Not Available")
    df["Start Hour1"] = df["Start Hour1"].fillna("Not Available")
    df["End Hour1"] = df["End Hour1"].fillna("Not Available")
    df["Language of Instruction"] = df["Language of Instruction"].fillna("English")
    
    df["Prerequisite"] = df["Prerequisite"].fillna("Not Available")
    df["Prerequisite"] = df["Prerequisite"].astype(str)
    df["Prerequisite"] = df["Prerequisite"].apply(lambda x: ",".join(re.findall(r'\d{7}', x)))
    df["Prerequisite"] = df["Prerequisite"].str.replace("", "Not Available")
    


    df["Course Objectives"] = df["Course Objectives"].fillna("Not Available")
    df["Course Objectives"] = df["Course Objectives"].str.replace("PLEASE VISIT THE COURSE WEBSITE AND COME TO THE FIRST HOURTHE INFORMATION HERE IS OUTDATED", "")
    df["Course Objectives"] = df["Course Objectives"].str.replace("•              ", "")
        
    df["Course Content"] = df["Course Content"].fillna("Not Available")
    df["Course Content"] = df["Course Content"].str.replace("Sorry no data is found for the course objective.", "Not Available")
    
    df["Course Learning Outcomes"] = df["Course Learning Outcomes"].fillna("Not Available")
    
    
    # Change order of columns
    new_column_order = ['Program Code', 'Program Short Name', 'Program Long Name', 'Course Code', 
                        'Credit', 'ECTS Credit', 'Course Section', 'Capacity', 'Language of Instruction', 
                        'Start Hour1', 'End Hour1', 'Day1', 'Instructor Name', 
                        'Level of Study', 'Prerequisite', 
                        'Course Objectives', 
                        'Course Content',
                        'Course Learning Outcomes']
    
    df = df[new_column_order]
    
    save_path = os.path.join(CLEANED_DATA_PATH, f"{code}-cleaned.csv")
    df.to_csv(save_path, sep=CSV_SEPARATOR, index=False)
    
    print(f"{file} is converted.")

902-merged.csv is converted.
905-merged.csv is converted.
908-merged.csv is converted.
911-merged.csv is converted.
903-merged.csv is converted.
906-merged.csv is converted.
910-merged.csv is converted.
901-merged.csv is converted.
904-merged.csv is converted.
909-merged.csv is converted.


In [8]:
cleaned_files = os.listdir(CLEANED_DATA_PATH)
cleaned_files

['903-cleaned.csv',
 '906-cleaned.csv',
 '910-cleaned.csv',
 '905-cleaned.csv',
 '909-cleaned.csv',
 '902-cleaned.csv',
 '908-cleaned.csv',
 'all-cleaned.csv',
 '901-cleaned.csv',
 '904-cleaned.csv',
 '911-cleaned.csv']

In [9]:
dataframes = []

for file in cleaned_files:
    path = os.path.join(CLEANED_DATA_PATH, file)
    df = pd.read_csv(path, sep=CSV_SEPARATOR)
    
    dataframes.append(df)


all_merged_df = pd.concat(dataframes, ignore_index=True)
all_merged_df.to_csv(os.path.join(CLEANED_DATA_PATH, "all-cleaned.csv"), index=False, sep=CSV_SEPARATOR)